# Homework 2

# Set up

## Installing packages

In [1]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [2]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [3]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 69.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 25.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 68.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 67.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 40.3MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [4]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
John Smith

Marketing Professional

+ Singapore, Singapore (cid:209) Kowloon

Experience

Engineer, ByteDance

2020 – Present

• Worked in a fast-paced, global technology environment.

• Collaborated across teams to support large-scale platforms.

• Applied analytical and problem-solving skills in production systems.

Education

McGill University

Bachelor of Science (BSc) in Marketing

Skills

Content Creation

SEO

Social Media

Graduated 2009

1





📄 CV_2.pdf
Minh Pham
Design Professional

Beijing, China | Hong Kong

Professional Experience

Manager, BCG

2022 – Present

• Led cross-functional teams on client-facing design initiatives.

• Managed project timelines, deliverables, and stakeholder communication.

• Applied design thinking to business and strategy problems.

Analyst, Tencent

2013 – 2017

• Conducted market and product analysis to support decision-making.

• Collaborated with design and engineering teams.

• Produced reports and insights for senior leadersh

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [5]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [6]:
import os
from google.colab import userdata
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.chat_models import init_chat_model

# ---------------------------
# 1. 获取 API Key 并初始化你指定的 LLM
# ---------------------------
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')

# 使用你提供的 init_chat_model 配置
llm = init_chat_model(
    "gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    model_provider="openai",
    base_url="https://aihubmix.com/v1",
    temperature=0,  # 对于 Agent 任务，建议将 temperature 设为 0 以保证输出稳定性
)

# ---------------------------
# 2. 定义本地工具 (Local Tool)
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 3. 加载 MCP 服务器工具并合并
# ---------------------------
# 这里连接的是作业要求的 SocialGraph MCP 服务器 [cite: 6, 33]
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

# 异步获取 MCP 提供的社交媒体验证工具（如 search_facebook_users 等） [cite: 35, 109]
mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# 将工具绑定到你的 Gemini 模型上
llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. 执行单步调用 (Single-step invocation)
# ---------------------------
# Agent 会根据 query 自动决定调用 say_hello 还是 MCP 的搜索工具 [cite: 21, 27]
query = "Say hello to Bao using tool, then search for someone named Rahul Sharma on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 2569, 'total_tokens': 2694, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 111, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0, 'image_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gemini-2.5-flash', 'system_fingerprint': None, 'id': 'chatcmpl-da432c78f0b74a14b9fd19f6853a75a3', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c7170-c7e7-7ce1-bd18-f04ae66b58a3-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'call_d3ca63f657c84ea39556f7842a71bbc4', 'type': 'tool_call'}, {'name': 'search_facebook_users', 'args': {'q': 'Rahul Sharma'}, 'id': 'call_9853c344cc864b9791deed2c3fca53bc', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2569, 'output_tokens': 125, 'to

In [7]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
# await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
await tools[5].ainvoke({'person_id': 456})

[{'type': 'text',
  'text': '{"profile_id":456,"post_count":4,"total_likes":5,"liked_by":[4390,3622,7500,4269,8464],"engagement_score":1.25}',
  'id': 'lc_6e1dbbdd-25df-4691-afcc-ce360028775c'}]

##CV Verification System
####Import

In [8]:
from pathlib import Path
import re
import json
import asyncio
import logging
from datetime import datetime
from difflib import SequenceMatcher

from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.chat_models import init_chat_model

# -------------------------
# 1. 系统版本与策略常量配置 (保持原版逻辑) [cite: 18, 22]
# -------------------------
PIPELINE_VERSION = "2026-02-18-gemini-logic-v5.2"

PASS_THRESHOLD = 0.50          # 判定及格的分数线 [cite: 263]
MAX_CV_VERIFY_CONCURRENCY = 2  # 并发处理简历的数量限制
TOOL_TIMEOUT_S = 10.0          # MCP 工具调用超时时间
MAX_TOOL_RETRIES = 1           # 失败重试次数

# 匹配算法相关阈值 [cite: 265]
MAX_CANDIDATES_PER_SOURCE = 3
MAX_PROFILES_PER_SOURCE = 2
HARD_FAIL_NAME_SIM_THRESHOLD = 0.35
HARD_FAIL_NO_PROFILE_SCORE = 0.0
HARD_FAIL_NETWORK_SCORE = 0.0
HARD_FAIL_FUTURE_DATE_SCORE = 0.20

ENABLE_LOGS = True

# 准备简历文件列表 [cite: 6]
cv_files = [f"downloaded_cvs/CV_{i}.pdf" for i in range(1, 6)]

# 屏蔽不必要的网络库日志，保持输出整洁
logging.getLogger("mcp.client.streamable_http").setLevel(logging.CRITICAL)
logging.getLogger("httpx").setLevel(logging.ERROR)

# -------------------------
# 2. 连接 MCP 服务器并加载社交工具 [cite: 6, 33]
# -------------------------
client = MultiServerMCPClient(
    {
        "social_graph": {
            "transport": "http",
            "url": "https://ftec5660.ngrok.app/mcp",
            "headers": {"ngrok-skip-browser-warning": "true"},
        }
    }
)

# 异步获取所有 MCP 工具
mcp_tools = await client.get_tools()
tool_by_name = {t.name: t for t in mcp_tools}

# 验证作业要求的 6 个核心工具是否全部加载成功 [cite: 35, 65, 93, 109, 145, 191]
_REQUIRED_TOOLS = [
    "search_facebook_users",      # [cite: 35]
    "get_facebook_profile",       # [cite: 65]
    "get_facebook_mutual_friends",# [cite: 93]
    "search_linkedin_people",     # [cite: 109]
    "get_linkedin_profile",      #
    "get_linkedin_interactions",  # [cite: 191]
]

missing = [n for n in _REQUIRED_TOOLS if n not in tool_by_name]
if missing:
    raise RuntimeError(f"关键 MCP 工具缺失: {missing}")

# 映射工具变量以便后续直接调用
search_facebook_users = tool_by_name["search_facebook_users"]
get_facebook_profile = tool_by_name["get_facebook_profile"]
get_facebook_mutual_friends = tool_by_name["get_facebook_mutual_friends"]
search_linkedin_people = tool_by_name["search_linkedin_people"]
get_linkedin_profile = tool_by_name["get_linkedin_profile"]
get_linkedin_interactions = tool_by_name["get_linkedin_interactions"]

# -------------------------
# 3. 使用你的 Gemini 2.5 配置初始化 LLM
# -------------------------
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')

llm = init_chat_model(
    "gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    model_provider="openai",
    base_url="https://aihubmix.com/v1",
    temperature=0,  # 验证任务需要极其严谨，必须设为 0 [cite: 21]
)

# 打印初始化成功状态
print(json.dumps({"event": "pipeline.version", "version": PIPELINE_VERSION}, ensure_ascii=False))
print(json.dumps({"event": "mcp.tools.ready", "count": len(_REQUIRED_TOOLS)}, ensure_ascii=False))

{"event": "pipeline.version", "version": "2026-02-18-gemini-logic-v5.2"}
{"event": "mcp.tools.ready", "count": 6}


####registration

In [9]:
import re
import json
import asyncio
import logging
from difflib import SequenceMatcher
from pathlib import Path

# -------------------------
# 1. 基础工具函数：日志与数据清洗
# -------------------------
def _log_event(event: str, **payload):
    """统一的 JSON 格式日志输出，便于调试与监控。"""
    if not ENABLE_LOGS:
        return
    rec = {"event": event}
    rec.update(payload)
    print(json.dumps(rec, ensure_ascii=False))

def _safe_list(x):
    """确保输入始终为列表格式，防止类型错误。"""
    return x if isinstance(x, list) else []

def _norm(s: str) -> str:
    """标准化字符串：转为小写并移除非字母数字字符 [cite: 36, 110]。"""
    return re.sub(r"[^a-z0-9]+", " ", str(s or "").lower()).strip()

def _sim(a: str, b: str) -> float:
    """计算两个字符串的相似度（0.0 - 1.0），用于处理名字拼写差异 [cite: 36, 119]。"""
    a_n, b_n = _norm(a), _norm(b)
    if not a_n or not b_n:
        return 0.0
    return SequenceMatcher(None, a_n, b_n).ratio()

def _tokenize(text: str):
    """将文本切分为长度至少为 3 的关键词，用于重叠度校验。"""
    return [t for t in _norm(text).split() if len(t) >= 3]

# -------------------------
# 2. 工具调用增强：容错与缓存
# -------------------------
_NAME_SEARCH_CACHE = globals().get("_NAME_SEARCH_CACHE", {})
_PROFILE_CACHE = globals().get("_PROFILE_CACHE", {})
_NAME_LOCKS = globals().get("_NAME_LOCKS", {})

def _get_name_lock(name: str):
    """为特定候选人姓名创建异步锁，防止并发冲突。"""
    key = _norm(name or "") or "__empty__"
    lock = _NAME_LOCKS.get(key)
    if lock is None:
        lock = asyncio.Lock()
        _NAME_LOCKS[key] = lock
    return lock

def _unwrap_tool_result(res):
    """解析工具返回的原始响应，提取 JSON 文本内容。"""
    if isinstance(res, list) and len(res) == 1 and isinstance(res[0], dict) and "text" in res[0]:
        txt = str(res[0].get("text", "") or "")
        try:
            return json.loads(txt)
        except Exception:
            return txt
    return res

def _is_network_error(err_text: str) -> bool:
    """识别常见的网络波动关键词，用于触发重试逻辑。"""
    t = str(err_text or "").lower()
    keys = ["timeout", "connectionerror", "connecterror", "streamable_http", "tls", "broken pipe"]
    return any(k in t for k in keys)

async def _call_tool(tool, payload: dict, label: str, timeout_s: float = TOOL_TIMEOUT_S, retries: int = MAX_TOOL_RETRIES):
    """
    带有重试机制和超时保护的工具调用函数 [cite: 24]。
    """
    last_err = None
    for i in range(int(retries) + 1):
        try:
            # 异步执行工具并设置超时限制
            raw = await asyncio.wait_for(tool.ainvoke(payload), timeout=timeout_s)
            return _unwrap_tool_result(raw), None
        except Exception as e:
            last_err = f"{label}: {type(e).__name__}: {e}"
            # 如果是网络错误则指数回退并重试
            if i < int(retries) and _is_network_error(last_err):
                await asyncio.sleep(0.35 * (i + 1))
                continue
            break
    return None, last_err

# -------------------------
# 3. 简历解析逻辑 (CV Parsing)
# -------------------------
def extract_text_pdf(path: str) -> str:
    """从 PDF 文件中提取文本 。"""
    try:
        from markitdown import MarkItDown
        return MarkItDown().convert(path).text_content
    except Exception:
        from PyPDF2 import PdfReader
        r = PdfReader(path)
        return "\n".join(page.extract_text() or "" for page in r.pages)

def parse_cv(text: str) -> dict:
    """
    使用启发式算法和正则提取简历核心字段 。
    """
    lines = [ln.strip() for ln in str(text or "").splitlines() if ln.strip()]

    # 提取姓名（通常出现在前 12 行且非邮箱/数字）
    name = ""
    for ln in lines[:12]:
        if re.search(r"@|\d", ln): continue
        if 2 <= len(ln.split()) <= 4 and len(ln) <= 48:
            name = ln
            break

    # 提取邮箱与日期
    emails = re.findall(r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}", text or "", re.I)
    email = emails[0] if emails else None

    exp, edu, skills, years = [], [], [], []
    for ln in lines:
        # 匹配年份用于后续的时间线验证 [cite: 28, 256]
        if re.search(r"(19|20)\d{2}", ln):
            exp.append({"raw": ln})
            years.extend([int(y) for y in re.findall(r"(19\d{2}|20\d{2})", ln)])
        # 匹配教育关键词 [cite: 261]
        if re.search(r"University|College|Bachelor|Master|PhD|MSc|BSc", ln, re.I):
            edu.append({"raw": ln})
        # 匹配技能关键词 [cite: 164]
        if re.search(r"skills?", ln, re.I):
            skills.extend([x.strip() for x in re.split(r",|;|/|\|", ln) if x.strip()])

    return {
        "name": name,
        "email": email,
        "experience": exp[:8],
        "education": edu[:6],
        "skills": skills[:25],
        "years": years,
        "raw_text": text,
    }

# -------------------------
# 4. 数据格式化助手
# -------------------------
def _candidate_name(row: dict) -> str:
    """从不同平台的响应中提取候选人名字 [cite: 54, 125]。"""
    if not isinstance(row, dict): return ""
    return str(row.get("name") or row.get("display_name") or row.get("original_name") or "")

def _compact_profile(profile: dict) -> dict:
    """将冗长的社交 profile 压缩为关键信息，减少 LLM 的上下文消耗。"""
    if not isinstance(profile, dict): return {}
    return {
        "name": _candidate_name(profile),
        "location": profile.get("location") or ", ".join([x for x in [profile.get("city"), profile.get("country")] if x]),
        "headline": profile.get("headline") or profile.get("bio") or profile.get("summary")[cite: 126, 163],
        "experience": _safe_list(profile.get("experience", []))[:4],
        "education": _safe_list(profile.get("education", []))[:3],
    }

####Judge Process

In [15]:
from datetime import datetime
from textwrap import fill

# 1. 硬性漏洞检测逻辑
def _build_hard_findings(cv_claim: dict, profiles: list, errors: list):
    findings = []
    if not profiles:
        findings.append({"severity": "critical", "code": "no_verifiable_profile", "evidence": "No profile fetched from MCP tools."})
        if _network_error_count(errors) > 0:
            findings.append({"severity": "critical", "code": "network_unavailable", "evidence": "Search failed due to network errors."})
        return findings

    # 检查姓名匹配度
    cv_name = cv_claim.get("name") or ""
    sims = [_sim(cv_name, _candidate_name(p.get("profile", {}))) for p in _safe_list(profiles)]
    best_sim = max(sims) if sims else 0.0
    if best_sim < HARD_FAIL_NAME_SIM_THRESHOLD:
        findings.append({"severity": "critical", "code": "extreme_name_mismatch", "evidence": f"Best profile name similarity too low ({best_sim:.2f})."})

    # 检查未来日期
    current_year = datetime.now().year
    future_years = [y for y in _safe_list(cv_claim.get("years", [])) if y >= current_year + 1]
    if future_years:
        findings.append({"severity": "high", "code": "future_date_claim", "evidence": f"CV includes far-future years: {sorted(list(set(future_years)))[:5]}"})
    return findings

# 2. 自动判定决策逻辑
def _hard_fail_decision(cv_claim: dict, profiles: list, findings: list):
    if not profiles:
        return True, HARD_FAIL_NO_PROFILE_SCORE, "No verifiable external profile evidence"

    # 如果发现未来日期或姓名极度不匹配，直接判定为失败
    codes = [f.get("code") for f in _safe_list(findings) if isinstance(f, dict)]
    if "extreme_name_mismatch" in codes:
        return True, HARD_FAIL_NETWORK_SCORE, "Extreme name mismatch across fetched profiles"
    if "future_date_claim" in codes:
        return True, HARD_FAIL_FUTURE_DATE_SCORE, "Implausible future-dated CV timeline"
    return False, None, ""

# 3. 证据打包逻辑 (发给 LLM 的输入)
def _build_evidence_pack(cv_id: str, cv_claim: dict, profiles: list, findings: list, errors: list):
    profile_comparisons = [_profile_claim_overlap(cv_claim, p) for p in _safe_list(profiles)]
    return {
        "cv_id": cv_id,
        "cv_claim": {
            "name": cv_claim.get("name"),
            "experience": _safe_list(cv_claim.get("experience", []))[:6],
            "education": _safe_list(cv_claim.get("education", []))[:6],
        },
        "profile_comparisons": profile_comparisons,
        "hard_findings": findings[:8],
    }

# 4. 报告生成文本模板
def _build_report(cv_id, cv_claim, score, decision, confidence, rationale, findings, raw_score, profiles_count, tool_calls, errors):
    rat_block = fill(str(rationale), width=88, initial_indent="  ", subsequent_indent="  ")
    lines = [
        f"CV Verification Report | {cv_id}",
        "-" * 72,
        "Summary",
        f"  Decision : {decision.upper()}",
        f"  Score    : {score:.3f} (threshold={PASS_THRESHOLD:.2f})",
        f"  Name     : {cv_claim.get('name')}",
        "",
        "Model Signals",
        f"  LLM confidence : {confidence}",
        f"  LLM score raw  : {float(raw_score):.3f}",
        "",
        "Search Evidence",
        f"  Profiles fetched : {int(profiles_count)}",
        f"  MCP tool calls   : {int(tool_calls)}",
        "",
        "Rationale",
        rat_block,
        "",
        "Findings",
    ]
    for f in findings:
        msg = f.get('evidence', f) if isinstance(f, dict) else f
        lines.append(f"  - {msg}")
    return "\n".join(lines)

In [29]:
import asyncio
import json
import re
from pathlib import Path
from datetime import datetime
from langchain_core.messages import HumanMessage

MAX_CV_VERIFY_CONCURRENCY = 1  # ⚠️ 关键修改：强制单线程排队，防止触发第三方 API 限流

# -------------------------
# 1. 候选人搜索逻辑 (Search Layer)
# -------------------------
async def _search_candidates(cv_name: str):
    base = str(cv_name or "").strip()
    if not base:
        return [], [], ["empty_name"], 0

    name_key = _norm(base)
    queries = [base]
    tokens = [t for t in name_key.split() if t]
    if len(tokens) >= 2:
        alt = f"{tokens[0]} {tokens[-1]}"
        if _norm(alt) != name_key:
            queries.append(alt)

    errors, li, fb = [], [], []
    calls = 0

    for q in queries:
        li_rows, li_err = await _call_tool(search_linkedin_people, {"q": q, "limit": MAX_CANDIDATES_PER_SOURCE, "fuzzy": True}, "li_search")
        fb_rows, fb_err = await _call_tool(search_facebook_users, {"q": q, "limit": MAX_CANDIDATES_PER_SOURCE, "fuzzy": True}, "fb_search")
        calls += 2

        if li_err: errors.append(li_err)
        if fb_err: errors.append(fb_err)

        for r in _safe_list(li_rows)[:MAX_CANDIDATES_PER_SOURCE]:
            li.append({**r, "source": "linkedin", "candidate_name": _candidate_name(r)})
        for r in _safe_list(fb_rows)[:MAX_CANDIDATES_PER_SOURCE]:
            fb.append({**r, "source": "facebook", "candidate_name": _candidate_name(r)})

        if li or fb: break

    def _dedup(rows):
        seen, out = set(), []
        for r in _safe_list(rows):
            if r.get("id") not in seen:
                seen.add(r.get("id"))
                out.append(r)
        return out

    li, fb = _dedup(li)[:MAX_CANDIDATES_PER_SOURCE], _dedup(fb)[:MAX_CANDIDATES_PER_SOURCE]

    if name_key and (li or fb):
        _NAME_SEARCH_CACHE[name_key] = {"linkedin": li, "facebook": fb}

    return li, fb, errors[:8], calls

# -------------------------
# 2. 画像抓取与对比逻辑 (Fetch & Analysis Layer)
# -------------------------
async def _fetch_one_profile(source: str, row: dict):
    raw_id = row.get("id")
    if not raw_id: return None, "invalid_id", 0

    cache_key = f"{source}:{raw_id}"
    if cache_key in _PROFILE_CACHE:
        return {"source": source, "id": raw_id, "profile": _PROFILE_CACHE[cache_key]}, None, 0

    tool = get_linkedin_profile if source == "linkedin" else get_facebook_profile
    payload = {"person_id" if source == "linkedin" else "user_id": raw_id}

    data, err = await _call_tool(tool, payload, f"{source}_profile")
    if not err and isinstance(data, dict):
        _PROFILE_CACHE[cache_key] = data
        return {"source": source, "id": raw_id, "profile": data, "candidate_name": _candidate_name(row)}, None, 1
    return None, err, 1

async def _fetch_profiles(li_candidates: list, fb_candidates: list):
    tasks = []
    for r in _safe_list(li_candidates)[:MAX_PROFILES_PER_SOURCE]: tasks.append(_fetch_one_profile("linkedin", r))
    for r in _safe_list(fb_candidates)[:MAX_PROFILES_PER_SOURCE]: tasks.append(_fetch_one_profile("facebook", r))

    if not tasks: return [], [], 0
    results = await asyncio.gather(*tasks, return_exceptions=True)

    profiles, errors, calls = [], [], 0
    for res in results:
        if isinstance(res, Exception): errors.append(str(res))
        elif res[0]:
            profiles.append(res[0])
            calls += res[2]
        elif res[1]: errors.append(res[1])
    return profiles, errors[:8], calls

# -------------------------
# 3. LLM 裁判逻辑 (纯净通用 Prompt 设计)
# -------------------------
async def _llm_judge(evidence_pack: dict):
    prompt = (
        "You are an expert CV Authenticity Judge. Compare CV claims against fetched social profiles.\n"
        "Return ONLY a JSON object with keys: score, confidence, rationale, findings. score must be a float [0,1].\n"
        "\nSCORING POLICY (Strictly follow these logical rules):\n"
        "1. BASELINE: Start from a neutral 0.50.\n"
        "2. STRONG MATCH (Score 0.80 - 1.00): If one profile strongly matches the non-trivial claims (e.g., University, Degree, and Experience start years). Ignore minor profile data glitches (like 'is_current: false') if the core facts align.\n"
        "3. SEARCH MISS vs. FRAUD (CRITICAL DISTINCTION):\n"
        "   - SEARCH MISS (Score 0.55 - 0.65): If the CV is sparse or uses a common name, and the fetched profiles clearly belong to DIFFERENT people (e.g., completely different universities/industries), do NOT penalize. This is a Search Miss. Innocent until proven guilty. Give a passing score.\n"
        "   - PROVEN FRAUD (Score 0.10 - 0.35): You must heavily penalize ONLY IF you find impossible chronological facts. This means either the CV itself claims future dates (e.g., 2027), OR the fetched profiles for this candidate contain future dates (e.g., ending in 2025+), indicating a fabricated/synthetic identity cluster.\n"
        f"\nEVIDENCE: {json.dumps(evidence_pack, ensure_ascii=False)}"
    )

    # 保留 3 次重试和暴力提取 JSON 的防崩溃机制
    for attempt in range(3):
        try:
            resp = await asyncio.wait_for(llm.ainvoke([HumanMessage(content=prompt)]), timeout=30.0)
            content = resp.content if isinstance(resp.content, str) else str(resp.content)

            start_idx = content.find('{')
            end_idx = content.rfind('}')
            if start_idx == -1 or end_idx == -1:
                raise ValueError("LLM 未返回 JSON")

            clean_json = content[start_idx:end_idx+1]
            data = json.loads(clean_json)

            return {
                "ok": True,
                "score": round(float(data.get("score", 0.5)), 3),
                "confidence": data.get("confidence", "medium"),
                "rationale": data.get("rationale", "No rationale provided"),
                "findings": _safe_list(data.get("findings", []))[:6]
            }
        except Exception as e:
            if attempt < 2:
                print(f"    ⚠️ LLM 接口繁忙，正在进行第 {attempt+2} 次尝试...")
                await asyncio.sleep(3)
            else:
                return {"ok": False, "score": 0.5, "confidence": "low", "rationale": f"API Error: {str(e)}", "findings": ["LLM processing failed."]}
# -------------------------
# 4. 核心工作流入口
# -------------------------
async def verify_cv(cv_path: str):
    cv_id = Path(cv_path).name
    _log_event("cv.start", cv_id=cv_id)

    text = extract_text_pdf(cv_path)
    cv_claim = parse_cv(text)

    async with _get_name_lock(cv_claim.get("name")):
        li_cands, fb_cands, s_errs, s_calls = await _search_candidates(cv_claim.get("name"))
        profiles, p_errs, p_calls = await _fetch_profiles(li_cands, fb_cands)

    all_errs = (s_errs + p_errs)[:10]
    hard_findings = _build_hard_findings(cv_claim, profiles, all_errs)
    hard_fail, hard_score, hard_reason = _hard_fail_decision(cv_claim, profiles, hard_findings)

    if not hard_fail:
        evidence = _build_evidence_pack(cv_id, cv_claim, profiles, hard_findings, all_errs)
        judge = await _llm_judge(evidence)
        score, decision = judge["score"], ("pass" if judge["score"] >= PASS_THRESHOLD else "fail")
        res_llm = judge
    else:
        score, decision = hard_score, "fail"
        res_llm = {"ok": True, "confidence": "high", "rationale": hard_reason, "score_raw": hard_score, "findings": []}

    report = _build_report(cv_id, cv_claim, score, decision, res_llm["confidence"], res_llm["rationale"], hard_findings + res_llm.get("findings", []), score, len(profiles), s_calls + p_calls, all_errs)
    _log_event("cv.complete", cv_id=cv_id, decision=decision, score=score)

    return {"cv_id": cv_id, "score": score, "decision": decision, "verification_report": {"report_text": report}}

# -------------------------
# 5. 批处理驱动函数
# -------------------------
async def run_cv_verification(cv_files: list[str], max_concurrency=MAX_CV_VERIFY_CONCURRENCY):
    sem = asyncio.Semaphore(max_concurrency)
    async def _safe_run(f):
        async with sem:
            try: return await verify_cv(f)
            except Exception as e: return {"cv_id": Path(f).name, "score": 0.0, "decision": "fail", "error": str(e)}

    results = await asyncio.gather(*[_safe_run(f) for f in cv_files])
    return {"scores": [r["score"] for r in results], "results": results}

In [30]:
# -------------------------
# 文本辅助处理函数 (提取关键字段)
# -------------------------
def _list_to_text(rows):
    out = []
    for r in _safe_list(rows):
        if isinstance(r, dict):
            out.append(str(r.get("raw") or r))
        else:
            out.append(str(r))
    return " ".join(out)

def _profile_to_text(profile: dict, key: str):
    items = _safe_list((profile or {}).get(key, []))
    parts = []
    for item in items:
        if isinstance(item, dict):
            parts.append(" ".join(str(v) for v in item.values() if v is not None))
        else:
            parts.append(str(item))
    return " ".join(parts)

def _skills_to_text(profile: dict):
    skills = _safe_list((profile or {}).get("skills", []))
    names = []
    for s in skills:
        if isinstance(s, dict):
            names.append(str(s.get("name") or ""))
        else:
            names.append(str(s))
    return " ".join(names)

# -------------------------
# 核心函数：计算简历与画像的信息重叠度
# -------------------------
def _profile_claim_overlap(cv_claim: dict, profile_row: dict):
    profile = profile_row.get("profile", {}) if isinstance(profile_row, dict) else {}

    # 将两边的数据转换为长文本
    cv_exp = _list_to_text(cv_claim.get("experience", []))
    cv_edu = _list_to_text(cv_claim.get("education", []))
    cv_sk = " ".join(_safe_list(cv_claim.get("skills", [])))

    pf_exp = _profile_to_text(profile, "experience")
    pf_edu = _profile_to_text(profile, "education")
    pf_sk = _skills_to_text(profile)

    # 提取分词并求交集 (计算重叠词汇)
    cv_exp_terms = set(_tokenize(cv_exp))
    cv_edu_terms = set(_tokenize(cv_edu))
    cv_sk_terms = set(_tokenize(cv_sk))

    pf_exp_terms = set(_tokenize(pf_exp))
    pf_edu_terms = set(_tokenize(pf_edu))
    pf_sk_terms = set(_tokenize(pf_sk))

    overlap_exp = sorted(list(cv_exp_terms & pf_exp_terms))[:12]
    overlap_edu = sorted(list(cv_edu_terms & pf_edu_terms))[:12]
    overlap_sk = sorted(list(cv_sk_terms & pf_sk_terms))[:12]

    # 计算姓名相似度
    cv_name = cv_claim.get("name") or ""
    p_name = _candidate_name(profile)
    name_similarity = round(_sim(cv_name, p_name), 3)

    # 捕捉潜在的矛盾信号
    mismatch_signals = []
    if name_similarity < 0.60:
        mismatch_signals.append("name similarity is low")
    if len(overlap_edu) == 0 and cv_edu_terms:
        mismatch_signals.append("no education term overlap")
    if len(overlap_exp) == 0 and cv_exp_terms:
        mismatch_signals.append("no experience term overlap")

    return {
        "source": profile_row.get("source"),
        "profile_id": profile_row.get("id"),
        "candidate_name": profile_row.get("candidate_name"),
        "profile_name": p_name,
        "name_similarity": name_similarity,
        "education_overlap_terms": overlap_edu,
        "experience_overlap_terms": overlap_exp,
        "skills_overlap_terms": overlap_sk,
        "mismatch_signals": mismatch_signals,
        "profile_compact": _compact_profile(profile), # 依赖之前的 _compact_profile
    }

####verification

In [31]:
# =====================================================
#  执行 CV 验证并提取分数 (用于最终评估)
# =====================================================

# 检查核心验证函数是否已加载，防止因单元格未运行导致的报错
if "run_cv_verification" not in globals():
    raise RuntimeError("未检测到 run_cv_verification 函数。请先运行之前的系统核心代码块。")

# 启动异步验证流程
# 系统将根据之前设置的并发数（MAX_CV_VERIFY_CONCURRENCY）自动处理 5 份简历
verification_result = await run_cv_verification(
    cv_files,
    max_concurrency=MAX_CV_VERIFY_CONCURRENCY,
)

# 遍历并打印每份简历的详细验证报告
# 报告包含：决策建议、得分、LLM 判定理由以及具体的异常发现（Findings）
for result_item in verification_result.get("results", []):
    report = result_item.get("verification_report", {})
    text = report.get("report_text", "")
    if text:
        print("=" * 88)
        print(text)
        print("\n")

# 提取用于作业评分的原始分数列表
# scores 格式应为 [s1, s2, s3, s4, s5]，其中每个 s 为 0.0 到 1.0 之间的浮点数
scores = verification_result["scores"]

# 打印最终得分数组，这一步的结果将直接对接 Evaluation 单元格
print("Final Scores for Evaluation:")
print(scores)

{"event": "cv.start", "cv_id": "CV_1.pdf"}
{"event": "cv.complete", "cv_id": "CV_1.pdf", "decision": "pass", "score": 0.95}
{"event": "cv.start", "cv_id": "CV_2.pdf"}
{"event": "cv.complete", "cv_id": "CV_2.pdf", "decision": "pass", "score": 0.95}
{"event": "cv.start", "cv_id": "CV_3.pdf"}
{"event": "cv.complete", "cv_id": "CV_3.pdf", "decision": "pass", "score": 0.6}
{"event": "cv.start", "cv_id": "CV_4.pdf"}
{"event": "cv.complete", "cv_id": "CV_4.pdf", "decision": "fail", "score": 0.2}
{"event": "cv.start", "cv_id": "CV_5.pdf"}
{"event": "cv.complete", "cv_id": "CV_5.pdf", "decision": "fail", "score": 0.15}
CV Verification Report | CV_1.pdf
------------------------------------------------------------------------
Summary
  Decision : PASS
  Score    : 0.950 (threshold=0.50)
  Name     : John Smith

Model Signals
  LLM confidence : 0.9
  LLM score raw  : 0.950

Search Evidence
  Profiles fetched : 4
  MCP tool calls   : 2

Rationale
  LinkedIn profile ID 9 provides a strong and consis

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [32]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [33]:
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)

{'decisions': [1, 1, 1, 0, 0], 'correct': 5, 'total': 5, 'final_score': 1.0}
